Import Libraries

In [ ]:
import os
import sys
from getpass import getpass
import torch.optim as optim
!pip install dropbox > /dev/null

Configure Environment

In [ ]:
os.environ["ENVIRONMENT"] = "lambda"
os.environ["REPO_DIR"] = "/home/ubuntu/UMARV-CV-ScenePerception"
os.environ["ROOT_DIR"] = "/home/ubuntu"
os.environ["MODEL_ID"] = "32o2bbqt"
os.environ["MODEL_DIR"] = f"{os.getenv('REPO_DIR')}/models/model_{os.getenv('MODEL_ID')}"

Configure git

In [ ]:
# Fill in your GitHub branch
git_branch = ""

In [ ]:
while not git_branch:
    git_branch = input("Enter your branch: ")

git_repo_url = "https://github.com/AwrodHaghiTabrizi/UMARV-CV-ScenePerception.git"
!git clone -b $git_branch $git_repo_url
%cd "{os.getenv('REPO_DIR')}"

Import Repository Resources

In [ ]:
sys.path.insert(0, f"{os.getenv('REPO_DIR')}/src")
from helpers import *

sys.path.insert(0, f"{os.getenv('MODEL_DIR')}/src")
from methods import *
from architecture import *
from dataset import *

Download Datasets

In [ ]:
dbx_access_token = getpass("Enter your DropBox access token: ")

In [ ]:
# Set to True if datasets need to be downloaded to your google drive
download_datasets = False

if download_datasets:
    download_datasets_from_dropbox(
        dbx_access_token = dbx_access_token,
        include_all_datasets = False,
        use_thread = True
    )

Code

In [5]:
num_epochs = 150
batch_size = 10
val_batch_size = 10
val_ratio = 0.2                         # Percent of training set used for validation
lookback = {"count": 0, "stride": 1}    # Prior frames model has access to

In [ ]:
device = set_device()
model = initialize_model(
    device = device,
    dbx_access_token = dbx_access_token,
    lookback = lookback,
    reset_weights = False
)

train_dataset, val_dataset = create_datasets(
    device = device,
    include_all_datasets = False,
    include_real_world_datasets = True,
    val_ratio = val_ratio,
    lookback = lookback
)

train_dataloader, val_dataloader = create_dataloaders(
    train_dataset = train_dataset,
    val_dataset = val_dataset,
    batch_size = batch_size,
    val_batch_size = val_batch_size
)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [ ]:
model, train_loss_hist, val_performance_hist, best_val_performance = training_loop(
    model = model,
    criterion = criterion,
    optimizer = optimizer,
    train_dataloader = train_dataloader,
    val_dataloader = val_dataloader,
    val_dataset = val_dataset,
    dbx_access_token = dbx_access_token,
    num_epochs = num_epochs,
    critiqueing_metric = "Accuracy",
    upload_weights_to_dropbox = True,
    auto_stop = False,
    verbose = True,
    display_sample_results = False
)

In [ ]:
graph_loss_history(train_loss_hist)

graph_performance_history(
    performance_hist = val_performance_hist,
    split = "Val",
    metrics = ["Accuracy", "Mean IoU"]
)

show_sample_results(
    model = model,
    dataset = val_dataset,
    device = device,
    num_samples = 2
)

In [ ]:
test_model_on_benchmarks(
    model = model,
    device = device,
    all_benchmarks = True,
    num_sample_results = 2,
    lookback = lookback
)

Push Changes (when ready)

In [ ]:
# Fill in GitHub user info
git_username = ""
git_email = ""

In [ ]:
while not git_username:
    git_username = input("Username left empty.\nGitHub username: ")
while not git_email:
    git_email = input("Email left empty.\nGitHub email: ")

!git config --global user.name $git_username
!git config --global user.email $git_email

git_access_token = getpass("Enter your GitHub access token: ")

git_push_url = f"https://{git_username}:{git_access_token}@{git_repo_url.replace('https://','')}"

commit_message = input("Commit message: ")
while not commit_message:
    commit_message = input("Commit message cannot be empty\nCommit message: ")

!git add .
!git commit -m "{commit_message}"
!git push $git_push_url $git_branch